In [3]:
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 17.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [4]:
#import bertopic
from bertopic import BERTopic

In [73]:
#import libraries
import pandas as pd
import numpy as np
import re
import string

#text-processing
import nltk
nltk.download('popular')
words = set(nltk.corpus.words.words())
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

#countvectorizer
from sklearn.feature_extraction.text import CountVectorizer

#c-tf-idf
from bertopic.vectorizers import ClassTfidfTransformer

#Dimension reduction
from umap import UMAP

#embedding
from sentence_transformers import SentenceTransformer

#HDBScan clustering
from hdbscan import HDBSCAN

#topic representation for fine tuning
from bertopic.representation import MaximalMarginalRelevance


[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

In [6]:
#import data from local drive
from google.colab import files
uploaded = files.upload()

Saving data.csv to data.csv


In [7]:
#load the data onto dataframe
import io
df = pd.read_csv(io.BytesIO(uploaded['data.csv']))

In [5]:
df.head()

,author_id,username,author_location,text,created_at
0,3345194367,Corix_JC,"London, England",The Future of Work and the Great Realignment \...,2022-10-30 22:05:01+00:00
1,1309334978795372544,_Jose_Rico,Dusseldorf,Learn more about the importance of agency coll...,2022-10-30 20:00:36+00:00
2,614598187,swetaregmi,Canada,#returntowork ?\n are #scentpolicy in place? \...,2022-10-30 19:00:16+00:00
3,130533518,chpar22,NaN,RT @mtptproject: #KITdays for coaching! \n\nCo...,2022-10-30 16:50:47+00:00
4,736472070,corixpartners,"London, UK",RT @Corix_JC: #Hybrid Work Is Not the Problem ...,2022-10-30 15:51:26+00:00


In [8]:
#check on the NULL values
df.isnull().sum()

author_id             0
username              0
author_location    2164
text                  0
created_at            0
dtype: int64

In [9]:
#replace null value with empty spaces
df2 = df.copy()

df2.fillna(' ', inplace=True)
df2.head()

,author_id,username,author_location,text,created_at
0,3345194367,Corix_JC,"London, England",The Future of Work and the Great Realignment \...,2022-10-30 22:05:01+00:00
1,1309334978795372544,_Jose_Rico,Dusseldorf,Learn more about the importance of agency coll...,2022-10-30 20:00:36+00:00
2,614598187,swetaregmi,Canada,#returntowork ?\n are #scentpolicy in place? \...,2022-10-30 19:00:16+00:00
3,130533518,chpar22,,RT @mtptproject: #KITdays for coaching! \n\nCo...,2022-10-30 16:50:47+00:00
4,736472070,corixpartners,"London, UK",RT @Corix_JC: #Hybrid Work Is Not the Problem ...,2022-10-30 15:51:26+00:00


## Text processing

In [10]:
#remove special characters, punctuations,numbers
def rmv_punct_speci(text):
    text = re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|[^a-zA-z0-9\s]', '', text) #remove special characters
    text = re.sub('[%s]'% re.escape(string.punctuation),'', text) #remove punctuation
    text = re.sub(r'\d+', '', text) #remove numbers
    text.split() #remove whitespaces
    return text

df2['clean_text'] = df2['text'].apply(rmv_punct_speci)

In [11]:
#remove accented characters
def clean(text):
    text = " ".join(w for w in nltk.wordpunct_tokenize(text) #tokenization
         if w.lower() in words or not w.isalpha())
    return text


df2['clean_text'] = df2['clean_text'].apply(clean)

In [13]:
df2.head()

,author_id,username,author_location,text,created_at,clean_text
0,3345194367,Corix_JC,"London, England",The Future of Work and the Great Realignment \...,2022-10-30 22:05:01+00:00,The Future of Work and the Great Realignment b...
1,1309334978795372544,_Jose_Rico,Dusseldorf,Learn more about the importance of agency coll...,2022-10-30 20:00:36+00:00,Learn more about the importance of agency coll...
2,614598187,swetaregmi,Canada,#returntowork ?\n are #scentpolicy in place? \...,2022-10-30 19:00:16+00:00,are in place My
3,130533518,chpar22,,RT @mtptproject: #KITdays for coaching! \n\nCo...,2022-10-30 16:50:47+00:00,for coaching Covering the of and group coachin...
4,736472070,corixpartners,"London, UK",RT @Corix_JC: #Hybrid Work Is Not the Problem ...,2022-10-30 15:51:26+00:00,Hybrid Work Is Not the Problem but Remote Who ...


##Topic Modeling with BERTopic

In [14]:
#Embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

Need to initiate the dimensional reduction before embedding because document embedding usually has hundred of dimension. Clustering would not works wonder in high dimensional data.

In [70]:
# Initiate UMAP
umap_model = UMAP(n_neighbors=10,
                  n_components=4,
                  min_dist=0.05,
                  metric='cosine',
                  random_state=100)


#Initiate HDBSCAN clustering
hdbscan_model = HDBSCAN(min_cluster_size=50,
                        gen_min_span_tree=True,
                        metric='euclidean',
                        prediction_data=True)

In [74]:
stopwords = list(stopwords.words('english')) + ['http', 'https', 'amp', 'com', 'rt', 'RT']

# Initiate the vectorizer model + stopwords to tokenize
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words=stopwords)

#c-TF-IDF
ctfidf_model = ClassTfidfTransformer()

#fine tuning topic representations
representation_model = MaximalMarginalRelevance(diversity=0.3)

In [75]:
# Initiate BERTopic
topic_model = BERTopic(nr_topics = 'auto',
                       min_topic_size = 25,
                       umap_model=umap_model,
                       embedding_model = embedding_model,
                       hdbscan_model = hdbscan_model,
                       vectorizer_model = vectorizer_model,
                       ctfidf_model = ctfidf_model,
                       representation_model = representation_model,
                       language="english",
                       calculate_probabilities=True)

# Run BERTopic model
topics, probabilities = topic_model.fit_transform(df2['clean_text'])

Topic 0 has the highest number of reviews with four terms such covid, pandemic, testing, and vaccination.

In [76]:
#extract the topics
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,5714,-1_office_work_return_business,"[office, work, return, business, workplace, re...",[FREE CUP OF JOE is back To welcome back our c...
1,0,486,0_workplace_covid testing_covid pandemic_covid...,"[workplace, covid testing, covid pandemic, cov...","[covid, covid, Covid]"
2,1,477,1_return work_career_recruitment_return,"[return work, career, recruitment, return, ret...",[Some from Return to work Recruitment Event Re...
3,2,420,2_team_back_office_back office,"[team, back, office, back office, lunch, welco...",[After the gap of two the Team is back in the ...
4,3,355,3_back office_return office_return_want return,"[back office, return office, return, want retu...","[Back to office, Back to Office, Back to Office]"
5,4,337,4_remote work_remote hybrid_work remote_hybrid...,"[remote work, remote hybrid, work remote, hybr...","[Remote Work That Matter To All Remote W, Remo..."
6,5,327,5_partner nope_like partner_right die_instead ...,"[partner nope, like partner, right die, instea...","[Cool, Indeed, Agreed]"
7,6,276,6_hybrid work_work model_hybrid working_hybrid...,"[hybrid work, work model, hybrid working, hybr...",[Explore critical for your strategy from our n...
8,7,252,7_workplace technology_innovation_flourishing_...,"[workplace technology, innovation, flourishing...","[workplace technology innovation, workplace te..."
9,8,246,8_para saber_em para_das_de em,"[para saber, em para, das, de em, de la, al, m...","[se a dos em as para a so do em e para saber, ..."


It seems we have lot of terms inside a topic, so lets see what are the top 10 terms for the selected topics.

In [77]:
# top 10 terms for topic 0
topic_model.get_topic(0)

[('workplace', 0.01517853196038907),
 ('covid testing', 0.015147345434394206),
 ('covid pandemic', 0.013580378665318943),
 ('covid covid', 0.009814398079161473),
 ('vaccination', 0.009714230876719968),
 ('back office', 0.009293629283050369),
 ('office covid', 0.009237080545093151),
 ('business covid', 0.009237080545093151),
 ('workplace covid', 0.007358974600580838),
 ('post pandemic', 0.007356089677108878)]

In [80]:
#Top 10 terms for topic 1
topic_model.get_topic(1)

[('return work', 0.031843152539450095),
 ('career', 0.02673210578372748),
 ('recruitment', 0.025294200905262885),
 ('return', 0.02381859790493931),
 ('returner', 0.019560894047424667),
 ('join', 0.015872702006738605),
 ('training', 0.015001948185846733),
 ('career break', 0.013348982416133803),
 ('back work', 0.012390054455264529),
 ('apply', 0.011985164654927341)]

In [81]:
# Top 10 terms for topic 2
topic_model.get_topic(2)

[('team', 0.042373013645049834),
 ('back', 0.02577797531552258),
 ('office', 0.022760292676797803),
 ('back office', 0.020631016651530876),
 ('lunch', 0.013847057052690742),
 ('welcome back', 0.013153773845340172),
 ('happy', 0.012337522063676417),
 ('celebrate', 0.010757862367766916),
 ('party', 0.009949313277498267),
 ('meeting', 0.009121735982668995)]

We will now visualize the top 10 topics presented, to understand better

In [78]:
# Visual the top 10 topics
topic_model.visualize_barchart(top_n_topics=10)

## Finding topic similarities

In [79]:
#check on the similarities between topics
topic_model.visualize_topics()